In [2]:
from atproto import Client
import os

client = Client()
profile = client.login(os.getenv('BSKY_USERNAME'), os.getenv('BSKY_PASSWORD'))

In [2]:
cursor = None
databs_posts = []

while True:
    fetched = posts = client.app.bsky.feed.search_posts(params={'q': '#databs', 'cursor': cursor})
    databs_posts = databs_posts + fetched.posts

    if not fetched.cursor:
        break

    cursor = fetched.cursor

In [3]:
unique_authors = list(set(post.author.handle for post in databs_posts))

In [ ]:
print(len(unique_authors))

In [5]:
def get_all_follows(author):
    cursor = None
    follows = []
    while True:
        fetched = client.app.bsky.graph.get_follows(params={'actor': author, 'cursor': cursor})
        follows = follows + fetched.follows
        if not fetched.cursor:
            break
        cursor = fetched.cursor
    return follows

In [ ]:
from tqdm import tqdm
import time

with open('databs.csv', 'w') as f:

    time.sleep(2)

    f.write("source,target,source_avatar_url,source_posts_count,source_followers_count,source_follows_count\n")

    for source in tqdm(unique_authors):

        try:
            author_follows = get_all_follows(source)
            source_actor = client.app.bsky.actor.get_profile(params={'actor': source})
        except Exception as e:
            print(f"Error processing author {source}: {str(e)}")
            continue

        for follow in author_follows:
            f.write(f"{source},{follow.handle},{source_actor.avatar},{source_actor.posts_count},{source_actor.followers_count},{source_actor.follows_count}\n")
